In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pprint import pprint
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Sales_test.csv')

In [3]:
data.head()

,Indx,Dt,Doc_no,Customer_id,Material_id,Plant_id,Quantity,Net,COGs,Month,Week_day,Weekend
0,0,1/11/2023,СД00-000001,"1,101,516",225014,7327,560.0,"195,794","163,162",1,3,рабочий
1,1,1/11/2023,СД00-000001,"1,101,516",228328,7327,560.0,"218,057","181,714",1,3,рабочий
2,2,1/11/2023,СД00-000006,"2,058,341",215889,7317,187.5,"25,714","21,428",1,3,рабочий
3,3,1/11/2023,СД00-000008,"1,101,385",244728,7317,90.0,"44,962","37,468",1,3,рабочий
4,4,1/11/2023,СД00-000008,"1,101,385",247866,7317,110.0,"44,248","36,873",1,3,рабочий


In [4]:
#Начинаем EDA

In [4]:
#Удаляем лишние данные
data_wodate = data.drop(columns=['Dt', 'Doc_no', 'Weekend', 'Indx'])
data

,Indx,Dt,Doc_no,Customer_id,Material_id,Plant_id,Quantity,Net,COGs,Month,Week_day,Weekend
0,0,1/11/2023,СД00-000001,"1,101,516",225014,7327,560.0,"195,794","163,162",1,3,рабочий
1,1,1/11/2023,СД00-000001,"1,101,516",228328,7327,560.0,"218,057","181,714",1,3,рабочий
2,2,1/11/2023,СД00-000006,"2,058,341",215889,7317,187.5,"25,714","21,428",1,3,рабочий
3,3,1/11/2023,СД00-000008,"1,101,385",244728,7317,90.0,"44,962","37,468",1,3,рабочий
4,4,1/11/2023,СД00-000008,"1,101,385",247866,7317,110.0,"44,248","36,873",1,3,рабочий
...,...,...,...,...,...,...,...,...,...,...,...,...
44703,44703,8/29/2023,СД00-011796,"1,111,671",233093,7337,1250.0,"190,054","158,378",8,2,рабочий
44704,44704,8/29/2023,СД00-011797,"1,115,838",228328,7337,700.0,"295,814","246,512",8,2,рабочий
44705,44705,8/29/2023,СД00-011797,"1,115,838",232992,7337,125.0,"19,762","16,468",8,2,рабочий
44706,44706,8/29/2023,СД00-011798,"1,117,728",223911,7337,560.0,"222,882","185,735",8,2,рабочий


In [5]:
#Проверям на наличие NaN
np.any(data.isna().values)

False

In [6]:
def convert_to_int(string):
    return int(string.replace(',', ''))

data_wodate['Customer_id'] = data_wodate['Customer_id'].apply(convert_to_int)
data_wodate[' Net '] = data_wodate[' Net '].apply(convert_to_int)
data_wodate[' COGs '] = data_wodate[' COGs '].apply(convert_to_int)

In [7]:
data_wodate.dtypes


Customer_id      int64
Material_id      int64
Plant_id         int64
Quantity       float64
 Net             int64
 COGs            int64
Month            int64
Week_day         int64
dtype: object

In [8]:
data


,Indx,Dt,Doc_no,Customer_id,Material_id,Plant_id,Quantity,Net,COGs,Month,Week_day,Weekend
0,0,1/11/2023,СД00-000001,"1,101,516",225014,7327,560.0,"195,794","163,162",1,3,рабочий
1,1,1/11/2023,СД00-000001,"1,101,516",228328,7327,560.0,"218,057","181,714",1,3,рабочий
2,2,1/11/2023,СД00-000006,"2,058,341",215889,7317,187.5,"25,714","21,428",1,3,рабочий
3,3,1/11/2023,СД00-000008,"1,101,385",244728,7317,90.0,"44,962","37,468",1,3,рабочий
4,4,1/11/2023,СД00-000008,"1,101,385",247866,7317,110.0,"44,248","36,873",1,3,рабочий
...,...,...,...,...,...,...,...,...,...,...,...,...
44703,44703,8/29/2023,СД00-011796,"1,111,671",233093,7337,1250.0,"190,054","158,378",8,2,рабочий
44704,44704,8/29/2023,СД00-011797,"1,115,838",228328,7337,700.0,"295,814","246,512",8,2,рабочий
44705,44705,8/29/2023,СД00-011797,"1,115,838",232992,7337,125.0,"19,762","16,468",8,2,рабочий
44706,44706,8/29/2023,СД00-011798,"1,117,728",223911,7337,560.0,"222,882","185,735",8,2,рабочий


In [9]:
data
data_wodate[['day', 'year']] = data['Dt'].str.split('/', expand=True)[[1, 2]]
# data_wodate = data_wodate.drop(columns=['day'])

In [10]:
df = data_wodate
df

,Customer_id,Material_id,Plant_id,Quantity,Net,COGs,Month,Week_day,day,year
0,1101516,225014,7327,560.0,195794,163162,1,3,11,2023
1,1101516,228328,7327,560.0,218057,181714,1,3,11,2023
2,2058341,215889,7317,187.5,25714,21428,1,3,11,2023
3,1101385,244728,7317,90.0,44962,37468,1,3,11,2023
4,1101385,247866,7317,110.0,44248,36873,1,3,11,2023
...,...,...,...,...,...,...,...,...,...,...
44703,1111671,233093,7337,1250.0,190054,158378,8,2,29,2023
44704,1115838,228328,7337,700.0,295814,246512,8,2,29,2023
44705,1115838,232992,7337,125.0,19762,16468,8,2,29,2023
44706,1117728,223911,7337,560.0,222882,185735,8,2,29,2023


In [11]:
#Разбиваем выборку на X и Y, так как ниже мы узнали что год 1 и тот же, удаляем его
X = df[[ 'Material_id', ' COGs ', 'Month', 'day']]

y = df['Quantity']
df[ 'Material_id'].describe()

count     44708.000000
mean     231610.866623
std       18671.329680
min      210023.000000
25%      223911.000000
50%      227395.000000
75%      234265.000000
max      909021.000000
Name: Material_id, dtype: float64

In [12]:
#Создаем модель для метода главных компонент для проверки на линейную зависимость между данными
model_PCA = PCA(n_components=X.shape[1])

model_PCA.fit(X)
model_PCA.explained_variance_ratio_

array([9.31334603e-01, 6.86653821e-02, 1.44247366e-08, 9.09696207e-10])

In [13]:
model_PCA.explained_variance_ratio_

array([9.31334603e-01, 6.86653821e-02, 1.44247366e-08, 9.09696207e-10])

In [14]:
#разделяем выборку на тестовую и тренировочную
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train = X[X['Month'] != 8]
y_train = y[X['Month'] != 8]
y_test = y[X['Month'] == 8]
X_test = X[X['Month'] == 8]
y_test

37569      20.0
37570      20.0
37571      50.0
37572     200.0
37573      40.0
          ...  
44703    1250.0
44704     700.0
44705     125.0
44706     560.0
44707     912.5
Name: Quantity, Length: 7139, dtype: float64

In [15]:
#создаем модель нс 
all_res = []
model = RandomForestRegressor(
    n_estimators=90, 
    max_depth=40, 
    min_samples_leaf=1, 
    min_samples_split=7, 
    max_leaf_nodes=197, 
    random_state=42
)
model.fit(X_train, y_train)
res = model.predict(X_test)
print(mean_squared_error(y_test, res))
print(mean_absolute_error(y_test, res))
r2_score(y_test, res)

7511.126984465183
15.251589339111467


0.9377301526855052

In [16]:
res = model.predict(X_test)
print(mean_squared_error(y_test, res))
print(mean_absolute_error(y_test, res))
r2_score(y_test, res)

7511.126984465183
15.251589339111467


0.9377301526855052

In [22]:
res = model.predict(X)
all_dt1 = pd.DataFrame({'trueres':data['Quantity'], 'Res':res, 'Date':data['Dt']})
all_dt1.to_csv('Res1.csv', index=False)

In [75]:
%pprint
#Включаем красивый вывод

Pretty printing has been turned ON


In [20]:
res = model.predict(X)
all_dt = pd.DataFrame({'Date':data['Dt'], 'Res':res, 'Material Id':df['Material_id']})
all_dt['Date'] = all_dt['Date'].str.replace('/', '.')
all_dt.to_csv('Res12.csv', index=False)